[Reference](https://python.plainenglish.io/the-lazy-loading-pattern-how-to-make-python-programs-feel-instant-31f1a3cfcbaf)

# Step 1: Move the Loading to Where It’s Actually Needed

In [3]:
def main():
    while True:
        print("1. Analyze sales")
        print("2. Count sales records")
        print("3. Quit")
        choice = input("Choose: ")

        if choice == "1":
            data = load_sales()  # Lazy load here
            print(analyze_sales(data))
        elif choice == "2":
            data = load_sales()  # Lazy load again
            print(count_sales(data))
        else:
            break

# Step 2: Combine Lazy Loading With Caching (Python Makes This Stupidly Easy)

In [4]:
from functools import cache

@cache
def load_sales():
    print("Loading sales data...")
    with open("big_sales.csv") as f:
        return list(csv.DictReader(f))

# Step 3: When Caching Gets Complicated (Remote APIs Say Hello)

In [5]:
import time

def ttl_cache(seconds):
    def decorator(func):
        cache_data = {}
        cache_time = {}
        def wrapper(*args):
            now = time.time()
            if (args in cache_data
                and now - cache_time[args] < seconds):
                return cache_data[args]
            result = func(*args)
            cache_data[args] = result
            cache_time[args] = now
            return result
        return wrapper
    return decorator

In [6]:
@ttl_cache(60)
def get_conversion_rates(currency):
    time.sleep(2)  # Simulate slow API
    return {"USD": 1.0, "EUR": 1.1, "JPY": 0.0065}

# Step 4: Lazy Streaming With Generators

In [7]:
def load_sales(path):
    with open(path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            yield row

In [8]:
def analyze_sales(path):
    total = 0
    for i, sale in enumerate(load_sales(path), start=1):
        total += float(sale["amount"])
        if i >= 10000:
            break
    return total

# Step 5: The Jedi Technique — Preloading in the Background

In [9]:
import threading

def preload_sales_data():
    def worker():
        load_sales()  # Cached
        count_sales()
    threading.Thread(target=worker, daemon=True).start()

In [10]:
preload_sales_data()  # Starts loading in background
show_menu()